# Pre-training

As in (Harsuko and Alkhalifah, 2022), pre-training of StorSeismic was intended to make the network learn the "features" of a seismic data. In this notebook, we will use the masked seismic data as the input and train the network to reconstruct the masked traces. The backbone of the network is BERT architecture (Devlin et al., 2018), obtained from HuggingFace's <code>transformers</code> library (https://github.com/huggingface/transformers).

## Load required modules

In [1]:
from transformers import BertConfig, BertForMaskedLM
import transformers
import torch.nn.functional as F
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from tqdm import tqdm
import os
import time
from radam import RAdam
import sys
import pandas as pd
import itertools

from storseismic.modules import *
from storseismic.train import run_pretraining
from storseismic.utils import *

from storseismic.utils import SSDataset
from torch.serialization import add_safe_globals

from storseismic.modules import PositionalEncoding
from transformers.models.bert.modeling_bert import BertForMaskedLM, BertModel, BertEmbeddings
from torch.nn.modules.linear import Linear
# Add these imports at the top with other imports
from torch.nn import Dropout, LayerNorm


pd.set_option('display.max_rows', None)
torch.cuda.empty_cache()

params = {
    'savefig.dpi': 600,  # to adjust notebook inline plot size
    'figure.dpi' : 600,
    'axes.labelsize':14,  # fontsize for x and y labels (was 10)
    'axes.titlesize':14,
    'axes.titleweight': 'bold',
    'legend.fontsize': 14,  # was 10
    'xtick.labelsize':12,
    'ytick.labelsize':12,
    'font.family': 'serif',
}
matplotlib.rcParams.update(params)

## Initialization

### Pre-training configuration

In [2]:
config = BertConfig()

# Model Parameter
config.hidden_size = 256
config.num_hidden_layers = 4
config.num_attention_heads = 4
config.num_hidden_ffn = 4
config.attention_type = "default"
config.k = 20
config.fixed = False
config.add_alibi = False # Add Linear Bias option (https://arxiv.org/abs/2108.12409)
config.alibi_type = "nosym"
config.fixed_slopes = False
config.add_urpe = False # Add Universal Relative Positional Encoding (https://arxiv.org/abs/2205.13401)

config.vocab_size = 2500 # Time samples
config.intermediate_size = config.hidden_size*config.num_hidden_ffn
config.max_length = 20 # Offsets
config.max_position_embeddings = config.max_length
config.position_embedding_type = 'sincos'
config.input_type = 'trace'
config.embedding_type = 'none'
config.type_vocab_size = 2
config.output_hidden_states = True
config.output_attentions = True
config.output_scores = True
# Toggle Pre-LN BERT
config.pre_ln = True # True, False

# Training Parameter
config.batch_size = 256
config.lr = 5e-4
config.epoch = 1000
config.patience = 20

# I/O parameter
config.parent_dir = './results/pretrain/'
config.dataset = './data/pretrain/'

# Create a fresh model
model = BertForMaskedLM(config).to(device)
print("Starting training from scratch with 2500 time dimension")

NameError: name 'device' is not defined

### Load data and wrap in Pytorch DataLoader

In [36]:
# Add SSDataset to the safe globals list
add_safe_globals([SSDataset])

train_data = torch.load(os.path.join(config.dataset, 'train_data.pt'))
test_data = torch.load(os.path.join(config.dataset, 'test_data.pt'))

vmin_all = -1
vmax_all = 1

In [37]:
batch_size = config.batch_size

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

for i, X in enumerate(train_dataloader):
    if i == 0:
        for j in range(4):
            f, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].imshow(X['inputs_embeds'][j, :, :].swapaxes(0, 1) - X['inputs_embeds'][j, :, :].mean(), 
                         aspect=.1, vmin=vmin_all, vmax=vmax_all, cmap='seismic')
            ax[0].set_title("Input")
            ax[1].imshow(X['labels'][j, :, :].swapaxes(0, 1) - X['labels'][j, :, :].mean(), 
                         aspect=.1, vmin=vmin_all, vmax=vmax_all, cmap='seismic')
            ax[1].set_title("Label")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
def analyze_pretraining_data(dataloader):
    """
    Analyze the pretraining data structure and content
    
    Args:
        dataloader: PyTorch DataLoader containing the training/test data
    """
    # Get first batch
    batch = next(iter(dataloader))
    
    print("Data Structure Information:")
    print("-" * 50)
    
    # Analyze inputs_embeds
    inputs = batch['inputs_embeds']
    print(f"Input Embeddings Shape: {inputs.shape}")
    print(f"Input Embeddings Type: {inputs.dtype}")
    print(f"Input Value Range: [{inputs.min():.3f}, {inputs.max():.3f}]")
    print(f"Input Mean: {inputs.mean():.3f}")
    print(f"Input Std: {inputs.std():.3f}")
    print()
    
    # Analyze labels
    labels = batch['labels']
    print(f"Labels Shape: {labels.shape}")
    print(f"Labels Type: {labels.dtype}")
    print(f"Labels Value Range: [{labels.min():.3f}, {labels.max():.3f}]")
    print(f"Labels Mean: {labels.mean():.3f}")
    print(f"Labels Std: {labels.std():.3f}")
    print()
    
    # Check for other keys in the batch
    print("Available keys in the data:")
    for key in batch.keys():
        print(f"- {key}")
    print()
    
    # Basic dataset size information
    print("Dataset Information:")
    print(f"Batch Size: {len(batch['inputs_embeds'])}")
    print(f"Total Batches: {len(dataloader)}")
    print(f"Total Samples: {len(dataloader.dataset)}")
    
    return batch

# Example usage:
print("Training Data Analysis:")
train_batch = analyze_pretraining_data(train_dataloader)
print("\nTest Data Analysis:")
test_batch = analyze_pretraining_data(test_dataloader)

# Visualize one example
def plot_example(batch, index=0):
    """
    Plot a single example from the batch
    
    Args:
        batch: Data batch dictionary
        index: Index of the example to plot (default: 0)
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Plot input
    im1 = ax1.imshow(
        batch['inputs_embeds'][index].T - batch['inputs_embeds'][index].mean(),
        aspect='auto',
        cmap='seismic',
        vmin=-1,
        vmax=1
    )
    ax1.set_title('Input Data')
    ax1.set_xlabel('Sequence Position')
    ax1.set_ylabel('Feature Dimension')
    plt.colorbar(im1, ax=ax1)
    
    # Plot label
    im2 = ax2.imshow(
        batch['labels'][index].T - batch['labels'][index].mean(),
        aspect='auto',
        cmap='seismic',
        vmin=-1,
        vmax=1
    )
    ax2.set_title('Label Data')
    ax2.set_xlabel('Sequence Position')
    ax2.set_ylabel('Feature Dimension')
    plt.colorbar(im2, ax=ax2)
    
    plt.tight_layout()
    plt.show()

# Plot an example
print("\nVisualizing an example from the training data:")
plot_example(train_batch)

Training Data Analysis:
Data Structure Information:
--------------------------------------------------
Input Embeddings Shape: torch.Size([256, 20, 2500])
Input Embeddings Type: torch.float32
Input Value Range: [-1.000, 1.000]
Input Mean: -0.000
Input Std: 0.129

Labels Shape: torch.Size([256, 20, 2500])
Labels Type: torch.float32
Labels Value Range: [-1.000, 1.000]
Labels Mean: -0.001
Labels Std: 0.096

Available keys in the data:
- inputs_embeds
- labels
- mask_label
- index

Dataset Information:
Batch Size: 256
Total Batches: 51
Total Samples: 12960

Test Data Analysis:
Data Structure Information:
--------------------------------------------------
Input Embeddings Shape: torch.Size([256, 20, 2500])
Input Embeddings Type: torch.float32
Input Value Range: [-1.000, 1.000]
Input Mean: -0.000
Input Std: 0.137

Labels Shape: torch.Size([256, 20, 2500])
Labels Type: torch.float32
Labels Value Range: [-0.657, 0.660]
Labels Mean: -0.000
Labels Std: 0.107

Available keys in the data:
- inputs

<IPython.core.display.Javascript object>

## Setup StorSeismic pre-training model

In [39]:
if not os.path.exists(config.parent_dir):
    os.makedirs(config.parent_dir)

### Swap built-in modules with the pre-defined ones

#### Swap the embedding and the prediction head

In [40]:
transformers.models.bert.modeling_bert.BertEmbeddings = BertEmbeddings
transformers.models.bert.modeling_bert.BertOnlyMLMHead = BertOnlyMLMHead
transformers.models.bert.modeling_bert.BertSelfAttention = BertSelfAttention

#### Check existing starting model before changing other modules

In [41]:
add_safe_globals([
    ('torch.nn.modules.dropout', 'Dropout'),
    ('torch.nn.modules.normalization', 'LayerNorm'),
    BertModel,
    Linear
])

starting_model_fn = ('_').join([str(config.hidden_size), str(config.num_hidden_layers), "startingmodel.pt"])
if not os.path.exists(os.path.join(config.parent_dir, "starting_model")):
    os.makedirs(os.path.join(config.parent_dir, "starting_model"))
if not os.path.exists(os.path.join(config.parent_dir, "starting_model", starting_model_fn)):
    model = BertForMaskedLM(config)
    torch.save(model, os.path.join(config.parent_dir, "starting_model", starting_model_fn))

#### Swap with Pre-LN or Post-LN

We will use Pre-LN BERT instead of Post-LN BERT, which is more insensitive to hyperparameter tuning (Xiong et al., 2020).

In [42]:
if config.pre_ln:
    transformers.models.bert.modeling_bert.BertSelfOutput = PreLNBertSelfOutput
    transformers.models.bert.modeling_bert.BertAttention = PreLNBertAttention
    transformers.models.bert.modeling_bert.BertIntermediate = PreLNBertIntermediate
    transformers.models.bert.modeling_bert.BertOutput = PreLNBertOutput

### Instantiate model

In [43]:
model = BertForMaskedLM(config)

#### Load weights from starting model (to preserve starting point on every run)

In [45]:
# Weight loading code with explicit handling for mismatched dimensions
starting_model_fn = "starting_bert_model.pt"
if os.path.exists(os.path.join(config.parent_dir, "starting_model", starting_model_fn)):
    # Load the pretrained model
    starting_model = torch.load(os.path.join(config.parent_dir, "starting_model", starting_model_fn), weights_only=False)
    pretrained_dict = starting_model.state_dict()
    
    # Get current model state dict
    model_dict = model.state_dict()
    
    # Filter out the mismatched size parameters
    filtered_dict = {k: v for k, v in pretrained_dict.items() 
                     if k in model_dict and v.shape == model_dict[k].shape}
    
    print(f"Found {len(filtered_dict)}/{len(pretrained_dict)} compatible weight tensors")
    
    # Handle special cases for LayerNorm
    layernorm_key_a = [key for key in filtered_dict.keys() if 'LayerNorm' in key and 'gamma' in key]
    layernorm_key_b = [key.replace('gamma', 'weight') for key in layernorm_key_a]
    for key_a, key_b in zip(layernorm_key_a, layernorm_key_b):
        filtered_dict[key_b] = filtered_dict.pop(key_a)
        
    layernorm_key_a = [key for key in filtered_dict.keys() if 'LayerNorm' in key and 'beta' in key]
    layernorm_key_b = [key.replace('beta', 'bias') for key in layernorm_key_a]
    for key_a, key_b in zip(layernorm_key_a, layernorm_key_b):
        filtered_dict[key_b] = filtered_dict.pop(key_a)
    
    # Update model dict with filtered weights
    model_dict.update(filtered_dict)
    
    # Load the filtered state dict
    model.load_state_dict(model_dict, strict=False)
    
    print("Successfully loaded compatible weights from pretrained model")
    print(f"Skipped loading {len(pretrained_dict) - len(filtered_dict)} incompatible tensors")
else:
    print("No pretrained weights found, starting training from scratch")

No pretrained weights found, starting training from scratch


#### Check number of trainable parameters

In [47]:
print(model)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(2500, 256, padding_idx=0)
      (position_embeddings): Embedding(20, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): PreLNBertAttention(
            (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): PreLNBertSelfOutput(
              (dense): Linear(in_features=256, out_features=

In [48]:
def count_parameters(model):
    table = pd.DataFrame(columns=['Name', 'Parameter'])
    total_params = 0
    i = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        total_params+=param
        table.loc[i] = [name] + [param]
        i += 1
    display(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(model)

,Name,Parameter
0,bert.embeddings.word_embeddings.weight,640000
1,bert.embeddings.position_embeddings.weight,5120
2,bert.embeddings.token_type_embeddings.weight,512
3,bert.embeddings.LayerNorm.weight,256
4,bert.embeddings.LayerNorm.bias,256
5,bert.encoder.layer.0.attention.LayerNorm.weight,256
6,bert.encoder.layer.0.attention.LayerNorm.bias,256
7,bert.encoder.layer.0.attention.self.query.weight,65536
8,bert.encoder.layer.0.attention.self.query.bias,256
9,bert.encoder.layer.0.attention.self.key.weight,65536


Total Trainable Params: 4447684


4447684

#### Send model to device

In [49]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
print("Using device: {}".format(device))

Using device: cuda


In [50]:
projection = nn.Linear(271, 256).to(device)


## Training

### Set up training hyperparameters

In [51]:
# Optimizer
optim = RAdam(model.parameters(), lr=config.lr)

# Loss
loss_fn = nn.MSELoss(reduction='mean')

# Number of epochs
epochs = config.epoch

### Train

In [52]:
%matplotlib notebook

plt.ion()
f, ax = plt.subplots(figsize=(4, 3))

<IPython.core.display.Javascript object>

In [53]:
# Make sure to define this in the notebook scope
projection = nn.Linear(2500, 256).to(device)

model, avg_train_loss, avg_valid_loss, time_per_epoch = run_pretraining(
    model, 
    optim, 
    loss_fn, 
    train_dataloader, 
    test_dataloader, 
    epochs, 
    device,
    projection,               # <--- Pass it here
    tmp_dir=config.parent_dir, 
    patience=config.patience,   
    plot=True, 
    f=f, 
    ax=ax
)

%matplotlib inline
plt.ioff()

  0%|          | 0/51 [00:00<?, ?it/s]/home/masa/miniconda3/envs/storseismic/lib/python3.13/site-packages/radam/radam.py:58: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value = 1) (Triggered internally at /pytorch/torch/csrc/utils/python_arg_parser.cpp:1661.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
Validation 0: 100%|██████████| 13/13 [00:01<00:00,  8.54it/s, loss=0.00417]


Epoch time: 8.55 s
Total time elapsed: 8.55 s
---------------------------------------
Validation loss decreased (inf --> 0.004146).  Saving model ...


Validation 1: 100%|██████████| 13/13 [00:01<00:00,  8.48it/s, loss=0.00177]


Epoch time: 7.97 s
Total time elapsed: 16.57 s
---------------------------------------
Validation loss decreased (0.004146 --> 0.001818).  Saving model ...


Validation 2: 100%|██████████| 13/13 [00:01<00:00,  8.42it/s, loss=0.00179]


...

Epoch time: 9.33 s
Total time elapsed: 496.51 s
---------------------------------------
EarlyStopping counter: 18 out of 20


Validation 57: 100%|██████████| 13/13 [00:01<00:00,  8.41it/s, loss=0.00148]


Epoch time: 9.08 s
Total time elapsed: 505.61 s
---------------------------------------
EarlyStopping counter: 19 out of 20


Validation 58: 100%|██████████| 13/13 [00:01<00:00,  7.23it/s, loss=0.00147]

Epoch time: 8.90 s
Total time elapsed: 514.53 s
---------------------------------------
EarlyStopping counter: 20 out of 20
Early stopping


### Plot results

In [ ]:
model.eval()

# Set dynamic extent parameters based on config
n_traces = config.max_position_embeddings    # Number of traces/channels
t_max = config.vocab_size * 8 / 1000         # Time dimension converted to seconds

idx = torch.randint(len(test_data), (4,))
print(idx)

inputs_embeds = test_data.encodings['inputs_embeds'][idx]
labels = test_data.encodings['labels'][idx]
mask_label = test_data.encodings['mask_label'][idx]

# Convert one-hot encoded inputs to token IDs if needed
# (this may not be needed for your SPECFEM data format)
input_ids = inputs_embeds.argmax(dim=-1) if inputs_embeds.shape[-1] > 256 else None

with torch.no_grad():
    # Choose the right input format based on your data
    if input_ids is not None:
        sample_output = model(input_ids=input_ids.to(device))
    else:
        # Project inputs to 256 dimensions
        inputs_256 = projection(inputs_embeds.float().to(device))
        sample_output = model(inputs_embeds=inputs_256)

# Updated plot for each sample
for i, (X, pred, label, mask) in enumerate(zip(inputs_embeds.cpu(),
                                             sample_output.logits.cpu(),
                                             labels.cpu(),
                                             mask_label.cpu())):
    
    f, ax = plt.subplots(1, 5, figsize=(20, 7.5), sharey=True, sharex=False)
    f.tight_layout()

    # IMPORTANT: Set aspect='auto' to handle the new dimensions
    # For the input data (X)
    X_transposed = X.T  # Shape becomes [time, channels]
    ax[0].imshow(X_transposed, aspect='auto', vmin=-1, vmax=1, cmap='seismic',
                extent=[0, n_traces, t_max, 0])
    ax[0].set_title("Input", fontsize=14)
    ax[0].xaxis.set_minor_locator(AutoMinorLocator(5))  
    ax[0].set_xlabel("Offset Index")
    ax[0].set_yticks(np.arange(0, t_max + 0.1, t_max/5))  # Create 5 ticks
    ax[0].yaxis.set_minor_locator(AutoMinorLocator(2))
    ax[0].set_ylabel("t (s)")
    
    # For the mask
    mask_transposed = mask.T  # Shape becomes [time, channels]
    ax[1].imshow(mask_transposed, aspect='auto', vmin=0, vmax=1, cmap='gray',
                extent=[0, n_traces, t_max, 0])
    ax[1].set_title("Mask", fontsize=14)
    ax[1].xaxis.set_minor_locator(AutoMinorLocator(5))
    ax[1].set_xlabel("Offset Index")
    
    # For the prediction
    pred_transposed = pred.T  # Shape becomes [time, channels]
    ax[2].imshow(pred_transposed, aspect='auto', vmin=-1, vmax=1, cmap='seismic',
                extent=[0, n_traces, t_max, 0])
    ax[2].set_title("Reconstructed", fontsize=14)
    ax[2].xaxis.set_minor_locator(AutoMinorLocator(5))
    ax[2].set_xlabel("Offset Index")
    
    # For the label
    label_transposed = label.T  # Shape becomes [time, channels]
    ax[3].imshow(label_transposed, aspect='auto', vmin=-1, vmax=1, cmap='seismic',
                extent=[0, n_traces, t_max, 0])
    ax[3].set_title("Label", fontsize=14)
    ax[3].xaxis.set_minor_locator(AutoMinorLocator(5))
    ax[3].set_xlabel("Offset Index")
    
    # For the difference
    diff = 10 * (label - pred)
    diff_transposed = diff.T  # Shape becomes [time, channels]
    im4 = ax[4].imshow(diff_transposed, aspect='auto', vmin=-1, vmax=1, cmap='seismic',
                    extent=[0, n_traces, t_max, 0])
    ax[4].set_title("10 x (Label - Reconstructed)", fontsize=14)
    ax[4].xaxis.set_minor_locator(AutoMinorLocator(5))
    ax[4].set_xlabel("Offset Index")
    
    plt.ylim(t_max, 0)
    
    cbar_ax = f.add_axes([0.92, 0.255, 0.0125, 0.52])
    cbar = f.colorbar(im4, cax=cbar_ax)
    cbar.set_ticks(np.arange(-1, 1.25, 0.25))
    
    plt.suptitle(f"Sample {i+1}", fontsize=16)
    plt.show()

tensor([ 847, 1665,  361, 1828])


## Finalize

In [57]:
# Save everything
print("Saving to", config.parent_dir)
if os.path.exists(os.path.join(config.parent_dir, 'model.pt')):
    if input("Path exists. Overwrite? (y/n)") == 'y':
        torch.save(model, os.path.join(config.parent_dir, 'model.pt'))
        avg_train_loss_arr = np.array(avg_train_loss)
        avg_valid_loss_arr = np.array(avg_valid_loss)
        np.save(os.path.join(config.parent_dir, 'train_loss.npy'), avg_train_loss_arr)
        np.save(os.path.join(config.parent_dir, 'valid_loss.npy'), avg_valid_loss_arr)
        torch.save(config, os.path.join(config.parent_dir, 'config.pt'))
        print("Saved successfully to", config.parent_dir)
    else:
        print("Saving failed.")
else:
    torch.save(model, os.path.join(config.parent_dir, 'model.pt'))
    avg_train_loss_arr = np.array(avg_train_loss)
    avg_valid_loss_arr = np.array(avg_valid_loss)
    np.save(os.path.join(config.parent_dir, 'train_loss.npy'), avg_train_loss_arr)
    np.save(os.path.join(config.parent_dir, 'valid_loss.npy'), avg_valid_loss_arr)
    torch.save(config, os.path.join(config.parent_dir, 'config.pt'))
    print("Saved successfully to", config.parent_dir)

Saving to ./results/pretrain/
Saved successfully to ./results/pretrain/


## References

Devlin, J., M.-W. Chang, K. Lee, and K. Toutanova, 2018, Bert: Pre-training of deep bidirectional transformers for language understanding: arXiv preprint arXiv:1810.04805.

Harsuko, R., and T. Alkhalifah, 2022, Storseismic: An approach to pre-train a neural network to store seismic data features: 83rd EAGE Annual Conference & Exhibition, European Association of Geoscientists & Engineers, 1–5.

Xiong, R., Yang, Y., He, D., Zheng, K., Zheng, S., Xing, C., Zhang, H., Lan, Y., Wang, L. and Liu, T., 2020, November. On layer normalization in the transformer architecture. In International Conference on Machine Learning (pp. 10524-10533). PMLR.